In [ ]:
import os
from glob import glob
import subprocess 
import json
from copy import copy, deepcopy

import networkx as nx

In [ ]:
ROOT = '../..'

In [ ]:
package_dirs = list(map(os.path.dirname, glob(os.path.join(ROOT, 'packages/pymedphys_*/src/pymedphys_*/__init__.py'))))
package_dirs

In [ ]:
internal_packages = [os.path.basename(directory) for directory in package_dirs]
internal_packages

In [ ]:
module_globs = {}

for package in internal_packages:
    module_globs[package] = list(
        map(os.path.dirname, glob(os.path.join(ROOT, 'packages', package, 'src', package, '*', '__init__.py'))))
    

module_globs

In [ ]:
glob('../../packages/pymedphys_fileformats/src/pymedphys_fileformats/trf/*.py')

In [ ]:
package_dirs = list(map(os.path.dirname, glob(os.path.join(ROOT, 'packages/pymedphys_*/src/pymedphys_logfiles/*/__init__.py'))))
package_dirs

In [ ]:
internal_packages = [os.path.basename(directory) for directory in package_dirs]
internal_packages

In [ ]:
dependencies_from_pydeps = {} 
for directory in package_dirs: 
    package = os.path.basename(directory) 
    dependencies_from_pydeps[package] = json.loads(subprocess.run(
        ["pydeps", directory, "--external"], stdout=subprocess.PIPE).stdout) 

In [ ]:
dependencies_set = { 
    package: {dependency for dependency in dependency_list if dependency in internal_packages} 
    for package, dependency_list in dependencies_from_pydeps.items() 
}
dependencies_set

In [ ]:
dag = nx.DiGraph()

for key, values in dependencies_set.items(): 
    dag.add_node(key) 
    dag.add_nodes_from(values) 
    edge_tuples = [ 
        (key, value) for value in values 
    ] 
    dag.add_edges_from(edge_tuples) 

assert nx.is_directed_acyclic_graph(dag)

In [ ]:
topological = list(nx.topological_sort(dag))
topological

In [ ]:
minimal_deps = {}

for package in topological[::-1]:
    if package in internal_packages:
        already_dependend_on = set()
        package_decendants = nx.descendants(dag, package)
        for dependency in package_decendants:
            if dependency in minimal_deps.keys():
                for already_in in nx.descendants(dag, dependency):
                    already_dependend_on.add(already_in)
        
        minimal_deps[package] = package_decendants.difference(already_dependend_on)
        
minimal_deps

In [ ]:
minimal_dag = nx.DiGraph()

for package in minimal_deps.keys():
    minimal_dag.add_node(package) 
    
for package, dependencies in minimal_deps.items(): 
    
    edge_tuples = [
        (package, dependency) for dependency in dependencies
    ]

    minimal_dag.add_edges_from(edge_tuples) 

In [ ]:
topological = list(nx.topological_sort(minimal_dag))
topological

In [ ]:
level_map = {}
for package in topological[::-1]:
    depencencies = nx.descendants(minimal_dag, package)
    levels = {0}
    for dependency in depencencies:
        try:
            levels.add(level_map[dependency])
        except KeyError:
            pass
    max_level = max(levels)
    level_map[package] = max_level + 1


level_map

In [ ]:
levels = {
    i + 1: []
    for i in range(max(level_map.values()))
}
for package, level in level_map.items():
    levels[level].append(package)
    
levels

In [ ]:
for level in levels.values():
    level.sort()

In [ ]:
clusters = ""

for i in range(max(level_map.values())):
    level = i + 1
    cluster_packages = ';\n        '.join(levels[level])
    clusters += """
    subgraph cluster_{} {{
        {};
        label = "Level {}";
        style = dashed;
        color = grey80;
    }}
    """.format(i, cluster_packages, level)
    
print(clusters)

In [ ]:
edges = ""

for edge in minimal_dag.edges():
    edges += "    {} -> {};\n".format(*edge)

print(edges)

In [ ]:
dot_file_contents = """
strict digraph  {{

    rankdir = LR;
    node [
        shape = box;
        width = 3;
    ];
    splines = polyline;
    
{}
{}
}}
""".format(clusters, edges)

print(dot_file_contents)